In [ ]:
# {r setup, include=FALSE}
#please do not touch this chunk
knitr::opts_chunk$set(echo = TRUE, results = "hold",fig.width = 7, fig.height = 4)
if(!require("pacman")) install.packages("pacman")
pacman::p_load(dplyr, ggplot2, plyr, tidyverse, pander, ggpubr, rapportools, knitr, pROC, reshape2)  

## Instruction

This exercise was adapted by Haoyue Shuai from one of her analysis on phenotype data in preparation for genetic association studies. Don't panic if you're just starting to learn R. The exercise does not involve writing serious software programs in R, but rather to execute some interactive and intuitive commands
to get some preliminary phenotype data analysis done. There will be some questions throughout that you can answer using your own words (and some data science intuition) with a few sentences. Some of them involve writing additional codes but you can mostly find and modify codes we provide as examples to address the questions. You do not have to answer all the questions but we strongly encourage you to make an attempt.

If you use Jupyter Notebook, please show your answers, codes and plots in the notebook when needed. If you work directly with this Rmd file and use R studio, please convert it into HTML format for us to review (simply by clicking the knit icon in your Rstudio toolbar). 
Please email the Jupyter Notebook or the Rmd + HTML files to us at wang.gao@columbia.edu. Please don't hesitate to contact us for any scientific or technical blockers, if you cannot find a solution yourself online.


### Dataset 

The data-set can be found as `UKB_Phenotype/data_cleaned.csv`.

## Genetic association study

We perform genetic association studies to identify genetic factors (variants) that may be involved in a complex trait (tinnitus, asthma, etc) etiology.
In brief, genetic association studies compare and identify difference in genetic data of individuals with disease (cases) to those without (controls). 
We report genetic variants that are observed more frequently in cases than in controls.

In order to perform genetic association studies, we need phenotype data and genotype data from individuals we collect.

- Phenotype and covariate data: age, sex, height, weight, condition for that trait (tinnitus in the example below case), etc.
- Genotype data: You can roughly understand it as a sequence of the bases in DNA molecules, A/T/C/G, for all chromosomes in human genome.

### Disease phenotype data 

We use a toy data-set extracted from the UK Biobank project.

**Load the data**

Load `UKB_Phenotype/data_cleaned.csv`. Note, you only need to complete the codes when seeing `YOUR CODE`. Please execute other existing codes as is.


In [2]:
# you need to put the data-set in the same folder
# where this .rmd file sits,
# which is here:
getwd()
sub_UKBB<-read.csv("UKB_Phenotype/data_cleaned.csv")

[1] "/home/shiya/orientation"

In [3]:
dim(sub_UKBB)

[1] 144756     11

In [4]:
colnames(sub_UKBB) # 11 variables 

[1] "IID"         "FID"         "f.4803.0.0"  "f.4803.1.0"  "f.4803.2.0" 
 [6] "f.4803.3.0"  "f.22001.0.0" "f.21003.0.0" "f.21003.1.0" "f.21003.2.0"
[11] "f.21003.3.0"

In [5]:
summary(sub_UKBB)

      IID               FID           f.4803.0.0         f.4803.1.0       
 Min.   :1000046   Min.   :1000046   Length:144756      Length:144756     
 1st Qu.:2267981   1st Qu.:2267981   Class :character   Class :character  
 Median :3527260   Median :3527260   Mode  :character   Mode  :character  
 Mean   :3521488   Mean   :3521488                                        
 3rd Qu.:4779335   3rd Qu.:4779335                                        
 Max.   :6025411   Max.   :6025411                                        
                                                                          
  f.4803.2.0         f.4803.3.0        f.22001.0.0         f.21003.0.0   
 Length:144756      Length:144756      Length:144756      Min.   :40.00  
 Class :character   Class :character   Class :character   1st Qu.:51.00  
 Mode  :character   Mode  :character   Mode  :character   Median :58.00  
                                                          Mean   :56.98  
                              

In [6]:
head(sub_UKBB)

,IID,FID,f.4803.0.0,f.4803.1.0,f.4803.2.0,f.4803.3.0,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>
1,1000046,1000046,NA,NA,"No, never",NA,Female,62,NA,73,NA
2,1000063,1000063,"No, never",NA,NA,NA,Male,43,NA,NA,NA
3,1000078,1000078,NA,"No, never","No, never",NA,Female,52,57,60,NA
4,1000105,1000105,"Yes, but not now, but have in the past",NA,NA,NA,Female,54,NA,NA,NA
5,1000112,1000112,NA,NA,"No, never",NA,Male,58,NA,68,NA
6,1000141,1000141,"No, never",NA,NA,NA,Female,49,NA,NA,NA


**Q1:** How many females and males are there in this data? Please show your code below how these numbers are computed.
 

In [7]:
table(sub_UKBB["f.22001.0.0"])


Female   Male 
 77535  67221 

**Q2:** Can you tell what kind of trait it is for tinnitus? 
A.Binary trait. B. Continuous trait. 

**Answer: A. Binary trait**

**Q3:** Recode f.4803

Field 4803 (f.4803) is the answers from participants for ACE touchscreen question "Do you get or have you had noises (such as ringing or buzzing) in your head or in one or both ears that lasts for more than five minutes at a time?" 

These fields contains answers to the questions in their first, 2nd, 3rd and 4th hospital visit: f.4803.0.0, f.4803.1.0, f.4803.2.0, f.4803.3.0. 


In [15]:
# Recode function:
df_recode<-function(df,column_name){
  new_names<-c()
  for (i in column_name){
    new_column_name<-paste0(i,"_recode")
    new_names<-c(new_names,new_column_name)
    df[,new_column_name] <- revalue(df[,i], c("No, never"= 0, 
                                            "Yes, but not now, but have in the past"= 1, 
                                            "Yes, now some of the time"= 1, 
                                            "Yes, now a lot of the time"= 1,
                                            "Yes, now most or all of the time"= 1,
                                            "Prefer not to answer"= NA,
                                            "Do not know"= NA ))
  }
  return (list(df=df,new_column_names=new_names))
}

# columns needs to be recoded:
column_name<-c("f.4803.0.0","f.4803.1.0","f.4803.2.0","f.4803.3.0")
# get a new data.frame with recoded columns added:
df_recode<-recode(df=sub_UKBB,column_name)$df
# get names of recoded columns:
new_column_names<-recode(df=sub_UKBB,column_name)$new_column_names
# show recode summary:
for (i in new_column_names)
{cat(i,"summary:");print(table(df_recode[,i]));cat("\n")}

f.4803.0.0_recode summary:
    0     1 
80805 31685 

f.4803.1.0_recode summary:
    0     1 
10282  4214 

f.4803.2.0_recode summary:
    0     1 
22583 11405 

f.4803.3.0_recode summary:
   0    1 
1451  691 



What do you think has been achieved by recoding these fields?

**Turn the information into a machine understandable format that's more suitable for model (e.g. regression**

**Q4:** Define case and control status of tinnitus for each participant in the study:

In [16]:
data_sub <- df_recode[,new_column_names]
# Function to define cases
f<-function(x){
  visit<-c()
  for (i in 1:4){
    if (!is.na(x[i]))
    {visit<-c(visit,x[i])}
  }
  if ("1" %in% visit){result= TRUE}
  else{result=FALSE}
  return (result)
}
# Apply the above function
df_recode$cases<-apply(data_sub, 1, f)
head(df_recode,10)

,IID,FID,f.4803.0.0,f.4803.1.0,f.4803.2.0,f.4803.3.0,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.4803.0.0_recode,f.4803.1.0_recode,f.4803.2.0_recode,f.4803.3.0_recode,cases
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<lgl>
1,1000046,1000046,NA,NA,"No, never",NA,Female,62,NA,73,NA,NA,NA,0,NA,FALSE
2,1000063,1000063,"No, never",NA,NA,NA,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE
3,1000078,1000078,NA,"No, never","No, never",NA,Female,52,57,60,NA,NA,0,0,NA,FALSE
4,1000105,1000105,"Yes, but not now, but have in the past",NA,NA,NA,Female,54,NA,NA,NA,1,NA,NA,NA,TRUE
5,1000112,1000112,NA,NA,"No, never",NA,Male,58,NA,68,NA,NA,NA,0,NA,FALSE
6,1000141,1000141,"No, never",NA,NA,NA,Female,49,NA,NA,NA,0,NA,NA,NA,FALSE
7,1000236,1000236,"No, never",NA,"No, never",NA,Male,61,NA,70,NA,0,NA,0,NA,FALSE
8,1000248,1000248,"No, never",NA,NA,NA,Male,63,NA,NA,NA,0,NA,NA,NA,FALSE
9,1000269,1000269,NA,NA,"No, never",NA,Male,60,NA,71,NA,NA,NA,0,NA,FALSE


How many cases and how many controls do we have for this phenotpype?

In [17]:
table(df_recode$cases)


 FALSE   TRUE 
101550  43206 

**Q5:** Extract a subset of columns from all participants for association study. 

In [18]:
df_cases  <- df_recode %>%
  select(IID,FID,cases)%>% 
  filter(cases==TRUE)
head(df_cases,10)

,IID,FID,cases
,<int>,<int>,<lgl>
1,1000105,1000105,TRUE
2,1000357,1000357,TRUE
3,1000443,1000443,TRUE
4,1000494,1000494,TRUE
5,1000625,1000625,TRUE
6,1000713,1000713,TRUE
7,1000728,1000728,TRUE
8,1000745,1000745,TRUE
9,1000875,1000875,TRUE


Please modify codes above to extract all the controls and keep only these columns: `FID`, `IID`, `cases`, `f.22001.0.0`, `f.21003.0.0`, `f.21003.1.0`, `f.21003.2.0`, `f.21003.3.0`. Please show the first 10 rows of the output.


In [19]:
df_cases  <- df_recode %>%
  select(IID,FID,cases,f.22001.0.0, f.21003.0.0, f.21003.1.0, f.21003.2.0, f.21003.3.0)%>% 
  filter(cases==TRUE)
head(df_cases,10)

,IID,FID,cases,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0
,<int>,<int>,<lgl>,<chr>,<int>,<int>,<int>,<int>
1,1000105,1000105,TRUE,Female,54,NA,NA,NA
2,1000357,1000357,TRUE,Female,69,NA,NA,NA
3,1000443,1000443,TRUE,Female,45,NA,52,NA
4,1000494,1000494,TRUE,Male,61,NA,NA,NA
5,1000625,1000625,TRUE,Female,60,NA,NA,NA
6,1000713,1000713,TRUE,Male,58,NA,67,NA
7,1000728,1000728,TRUE,Male,61,NA,NA,NA
8,1000745,1000745,TRUE,Male,60,NA,NA,NA
9,1000875,1000875,TRUE,Male,59,NA,NA,NA


**Q6:** Field 21003 contains the information of the age of participants, same as field 4803. Note that some of them have more than one age. Can you guess why?

**That's the age of each visit, so participants will get older with subsequent visits.**

**Q7:** For those with more than one age records, which age do you think should be used in the genetic association analysis?

**Depends on the exact question we're trying to answer, but probably the first one. There might be bias involved if we use subsequent visits, e.g. patients taking better care of themselves after the first visit etc. **

**Q8:** Please compute a summary of age information for controls (you can use `summary()` function in R):



In [26]:
age_controls  <- df_recode %>%
  select(cases,f.21003.0.0)%>% 
  filter(cases==FALSE)
summary(age_controls$f.21003.0.0)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  40.00   51.00   58.00   56.76   63.00   73.00 

### Association testing via regression analysis

To identify genetic factors that may be involved in this trait (tinnitus), we would need to find the association between the genotype and the phenotype of each individual. 
Regression analysis is the basis of many association analysis. Instead of overwhelming you with huge genotype data, we use here a simple dataset for regression analysis to demonstrate what association studies look like.


We fit below simple linear model with 2 variables from a data-set to see their relationship. For example `mpg` vs. `weight` in this Auto data-set. 

**Q9:** Is there association between `mpg of the car` and `weight of the car`? If so, it appearing to be positive or negative? Is the association significant and why? 

**Negative. It is significant since p-value for F-tests and individual t-tests are near 0**

In [27]:
# check if you have ISLR package, if not, install it
if(!requireNamespace('ISLR')) install.packages('ISLR') 
auto_data <- ISLR::Auto
#  fit a linear regression model
fit_1<-lm(mpg ~ weight, auto_data)
summary(fit_1)

Loading required namespace: ISLR




Call:
lm(formula = mpg ~ weight, data = auto_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-11.9736  -2.7556  -0.3358   2.1379  16.5194 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 46.216524   0.798673   57.87   <2e-16 ***
weight      -0.007647   0.000258  -29.64   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.333 on 390 degrees of freedom
Multiple R-squared:  0.6926,	Adjusted R-squared:  0.6918 
F-statistic: 878.8 on 1 and 390 DF,  p-value: < 2.2e-16


**Q10:** Please create a new variable to indicate cars having MPG greater than 23 as 1, otherwise 0, then use logistic regrssion via `glm()` function to analyze association between weight and this new variable you just created. Please comment on what you find.

**comments: the model and individual effects are still significant, although the interpretation of the estimates are different**

In [30]:
auto_data$mpg_binary <- as.integer(auto_data$mpg > 23)
fit_2<-glm(mpg_binary ~ weight, auto_data, family=binomial)
summary(fit_2)


Call:
glm(formula = mpg_binary ~ weight, family = binomial, data = auto_data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3700  -0.3330  -0.0277   0.4102   2.7685  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) 11.4448009  1.1350641  10.083   <2e-16 ***
weight      -0.0040954  0.0004115  -9.952   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 542.60  on 391  degrees of freedom
Residual deviance: 234.54  on 390  degrees of freedom
AIC: 238.54

Number of Fisher Scoring iterations: 6


**Q11:** Find the `Estimates` from your asociation results summary. How do you interpret the estimated effects of weight in the context of linear regression, and in the context of logististic regression? (this question might be a bit challenging if you are not familiar with regression analysis; don't sweat if you cannot find the answer for it).

**It means for every increase in 1 unit of "weight", there is a 0.0040954 decrease in the log odds of that observation having MPG > 23. Log odds defined to be ln(p/(1-p)) where p is probability of MPG > 23.**